In [85]:
import pandas as pd
import numpy as np
import glob
from matplotlib import pyplot as plt
from sklearn import metrics
from os.path import basename
pd.options.display.max_columns = 1200
pd.options.display.max_rows = 1200

# Read in and Prep Data

In [171]:
# df = pd.read_csv('data/autoperf_filtered_1000000ch', header=0).sample(frac=1)
df = pd.read_csv('../data/mira_djc_complete/dim_job_composite.csv', header=0).sample(frac=1)

# li = []
# for filename in glob.glob("/home/luckierdodge/repos/jupyter-notebooks/data/autoperf_complete/autoperf-201*.csv"):
#     frame = pd.read_csv(filename, header=0)
#     li.append(frame)
# df = pd.concat(li, axis=0, ignore_index=True)

# df.dropna(inplace=True);

In [172]:
print(df.shape)
df = df.dropna()
print(df.shape)

(276314, 58)
(275309, 58)


In [173]:
# counts = df.zero_execName.value_counts().to_dict()
# df_filtered = pd.DataFrame()
# for name in counts:
#     if counts[name] >= 500:
#         df_filtered = df_filtered.append(df[df["zero_execName"] == name])
# df = df_filtered.copy()

In [174]:
df['EXEC_NAME'] = df['COMMAND'].apply(basename)
print(len(df.COMMAND.unique()))
print(len(df.EXEC_NAME.unique()))

72299
33758


In [91]:
train_size = .8
train = df.head(int(len(df.index) * train_size))
test = df.tail(int(len(df.index) * (1-train_size)))

In [196]:
features = ["EXEC_NAME", "USERNAME", "PROJECT_NAME", "QUEUE_NAME"]
targets = ["WALLTIME_SECONDS"]
# targets = ["zero_numRanks"]

X_train = train[features]
X_test = test[features]
Y_train = train[targets]
Y_test = test[targets]

# Linear Regression using SciKit Learn (One-Hot Encoding)

In [233]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [235]:
# label_encoder = LabelEncoder()
enc = OneHotEncoder(handle_unknown='ignore', sparse=True, categories='auto')
enc.fit(X_train.values)

# One-hot Encode
# user_feature = label_encoder.fit_transform(X_train.values)
# user_feature = user_feature.reshape(len(train.index), 1)
# user_feature = onehot_encoder.fit_transform(X_train.values)

# user_feature = label_encoder.fit_transform(X_test.values)
# user_feature = user_feature.reshape(len(test.index), 1)
# user_feature = onehot_encoder.fit_transform(X_test.values)

X_train_one_hot = enc.transform(X_train.values)
X_test_one_hot = enc.transform(X_test.values)

In [218]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(X_train_one_hot, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [219]:
model.score(X_test_one_hot, Y_test)

0.789298131898288

In [220]:
import math
Y_predict = model.predict(X_test_one_hot)
regression_model_mse = metrics.mean_squared_error(Y_predict, Y_test)
math.sqrt(regression_model_mse)

6579.199413220353

In [221]:
print(math.sqrt(regression_model_mse) / (max(Y_test.values) - min(Y_test.values)))

[0.0303161]


# Keras Analysis

In [222]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [223]:
train_examples = X_train_one_hot
train_targets = Y_train
test_examples = X_test_one_hot
test_targets = Y_test

In [224]:
train_examples

<275309x35797 sparse matrix of type '<class 'numpy.float64'>'
	with 1101236 stored elements in Compressed Sparse Row format>

In [225]:
def batchLoader(train_examples, train_targets, batch_size):

    dataset_length = train_examples.shape[0]

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < dataset_length:
            limit = min(batch_end, dataset_length)
            X = train_examples[batch_start:limit]
            Y = train_targets[batch_start:limit]

            yield (X, Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size

In [226]:
g = batchLoader(train_examples, train_targets, 32)
next(g)

(<32x35797 sparse matrix of type '<class 'numpy.float64'>'
 	with 128 stored elements in Compressed Sparse Row format>,
         WALLTIME_SECONDS
 83839            43200.0
 220758            3600.0
 72403            43200.0
 36116              900.0
 271206            3600.0
 243521            6600.0
 253582           21600.0
 30051             5400.0
 114684           43200.0
 15838              600.0
 269505           21600.0
 233678           14400.0
 240416           21600.0
 154505            3600.0
 210694            5400.0
 115234             900.0
 183157            3600.0
 218144           21600.0
 230695           14400.0
 126298            4200.0
 248904            3600.0
 266295            6000.0
 51000             3600.0
 158409            3600.0
 149521            2400.0
 169640            8400.0
 249188            5400.0
 204130           21600.0
 52212             3600.0
 239546            1800.0
 16327             3600.0
 107331           21600.0)

In [227]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(train_examples.shape[1],)))
# model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(256, activation='relu',))
# model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(256, activation='relu',))
# model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1))

In [228]:
optimizer = tf.keras.optimizers.RMSprop(0.004)
model.compile(optimizer=optimizer,
             loss='mean_squared_error',
             metrics=['mean_absolute_error'],
             )

In [229]:
# model.fit(train_examples, train_targets, epochs=10)
model.fit(batchLoader(train_examples, train_targets, 64), epochs=10, steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 7s 72ms/step - loss: 143789835.5800 - mean_absolute_error: 6747.8184
Epoch 2/10
100/100 [==============================] - 6s 64ms/step - loss: 80485378.0200 - mean_absolute_error: 4770.7329
Epoch 3/10
100/100 [==============================] - 6s 64ms/step - loss: 71925576.3000 - mean_absolute_error: 4365.8032
Epoch 4/10
100/100 [==============================] - 6s 65ms/step - loss: 67267333.4600 - mean_absolute_error: 4185.5781
Epoch 5/10
100/100 [==============================] - 6s 64ms/step - loss: 59788526.5200 - mean_absolute_error: 3947.1753
Epoch 6/10
100/100 [==============================] - 6s 64ms/step - loss: 58521405.4200 - mean_absolute_error: 3804.2222
Epoch 7/10
100/100 [==============================] - 7s 66ms/step - loss: 53772306.2700 - mean_absolute_error: 3689.3303
Epoch 8/10
100/100 [==============================] - 6s 64ms/step - loss: 53119127.6900 - mean_absolute_error: 3683.4360
Epoch 9/10
100/100 [===

In [230]:
test_loss, test_abs_err = model.evaluate(batchLoader(test_examples, test_targets, 64), steps=100)

100/100 [==============================] - 2s 16ms/step - loss: 52677844.6050 - mean_absolute_error: 3431.4553


In [231]:
print(test_abs_err / 3600)

0.9531820339626736
